In [2]:
from pennylane import numpy as np
import pennylane as qml
from matplotlib import pyplot as plt

from functions_vqe_crx import vqe_hee, vqe_ng, vqe_hee_th
import numpy as np
from joblib import Parallel, delayed

device = 'cpu'

## AntiF Heisenberg hamiltonian definition 

In [4]:
n_cells = (2, 2)
coupling = [1,1,1]

qubits = n_cells[0] * n_cells[1]

lattice = 'triangle'

H = qml.spin.heisenberg(lattice, n_cells, coupling, boundary_condition=True)

H

(
    X(2) @ X(3)
  + Y(2) @ Y(3)
  + Z(2) @ Z(3)
  + X(1) @ X(3)
  + Y(1) @ Y(3)
  + Z(1) @ Z(3)
  + X(1) @ X(2)
  + Y(1) @ Y(2)
  + Z(1) @ Z(2)
  + X(0) @ X(2)
  + Y(0) @ Y(2)
  + Z(0) @ Z(2)
  + X(0) @ X(3)
  + Y(0) @ Y(3)
  + Z(0) @ Z(3)
  + X(0) @ X(1)
  + Y(0) @ Y(1)
  + Z(0) @ Z(1)
)

## Distribution of results

In [9]:
max_iterations = 100

layers = np.arange(2, 20, 4)
n_layers = len(layers)

n_runs = 100

conv_tol = 1e-06

weights_lr = 0.1
phi_lr = 0.05

In [10]:
def run_single_simulation_1(i, l, j):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_hee(H, qubits, l, lr=weights_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)
    
    return i, j, (energy, grad_norm, grad_var, conv)

def run_single_simulation_2(i, l, j):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_hee_th(H, qubits, l, weights_lr=weights_lr, phi_lr=phi_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)
    
    return i, j, (energy, grad_norm, grad_var, conv)

### HEE

In [12]:
tasks = (delayed(run_single_simulation_1)(i, l, j)
         for i, l in enumerate(layers)
         for j in range(n_runs))
print('number of tasks:', n_layers * n_runs)
results = Parallel(n_jobs=20, verbose=11)(tasks)

number of tasks: 500


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   13.1s
[Parallel(n_jobs=20)]: Done   2 tasks      | elapsed:   13.1s
[Parallel(n_jobs=20)]: Done   3 tasks      | elapsed:   13.1s
[Parallel(n_jobs=20)]: Done   4 tasks      | elapsed:   13.3s
[Parallel(n_jobs=20)]: Done   5 tasks      | elapsed:   13.4s
[Parallel(n_jobs=20)]: Done   6 tasks      | elapsed:   13.4s
[Parallel(n_jobs=20)]: Done   7 tasks      | elapsed:   13.4s
[Parallel(n_jobs=20)]: Done   8 tasks      | elapsed:   13.5s
[Parallel(n_jobs=20)]: Done   9 tasks      | elapsed:   13.7s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   13.7s
[Parallel(n_jobs=20)]: Done  11 tasks      | elapsed:   13.9s
[Parallel(n_jobs=20)]: Done  12 tasks      | elapsed:   14.0s
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:   14.0s
[Parallel(n_jobs=20)]: Done  14 tasks      | elapsed:   14.0s
[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:  

KeyboardInterrupt: 

In [ ]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_layers, n_runs, max_iterations+1))
grad_norms = np.zeros((n_layers, n_runs, max_iterations))
grad_variances = np.zeros((n_layers, n_runs, max_iterations))
convergences = np.zeros((n_layers, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]

In [ ]:
prefix = "../local_data/testlog0/"
spec = "_crx_1"

np.save(prefix+"energies"+spec+".npy", energies)
np.save(prefix+"grad_norms"+spec+".npy", grad_norms)
np.save(prefix+"grad_variances"+spec+".npy", grad_variances)
np.save(prefix+"convergences"+spec+".npy",convergences)

### NG

In [ ]:
tasks = (delayed(run_single_simulation_2)(i, l, j)
         for i, l in enumerate(layers)
         for j in range(n_runs))
print('number of tasks:', n_layers * n_runs)
results = Parallel(n_jobs=20, verbose=11)(tasks)

number of tasks: 1500


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   34.8s
[Parallel(n_jobs=20)]: Done   2 tasks      | elapsed:   34.8s
[Parallel(n_jobs=20)]: Done   3 tasks      | elapsed:   34.9s
[Parallel(n_jobs=20)]: Done   4 tasks      | elapsed:   35.4s
[Parallel(n_jobs=20)]: Done   5 tasks      | elapsed:   35.4s
[Parallel(n_jobs=20)]: Done   6 tasks      | elapsed:   35.9s
[Parallel(n_jobs=20)]: Done   7 tasks      | elapsed:   36.2s
[Parallel(n_jobs=20)]: Done   8 tasks      | elapsed:   36.3s
[Parallel(n_jobs=20)]: Done   9 tasks      | elapsed:   36.7s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   37.0s
[Parallel(n_jobs=20)]: Done  11 tasks      | elapsed:   37.1s
[Parallel(n_jobs=20)]: Done  12 tasks      | elapsed:   37.2s
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:   37.3s
[Parallel(n_jobs=20)]: Done  14 tasks      | elapsed:   37.3s
[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:  

KeyboardInterrupt: 

In [ ]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_layers, n_runs, max_iterations+1))
grad_norms = np.zeros((n_layers, n_runs, max_iterations))
grad_variances = np.zeros((n_layers, n_runs, max_iterations))
convergences = np.zeros((n_layers, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]

In [ ]:
prefix = "../local_data/testlog0/"
spec = "_crx_2"

np.save(prefix+"energies"+spec+".npy", energies)
np.save(prefix+"grad_norms"+spec+".npy", grad_norms)
np.save(prefix+"grad_variances"+spec+".npy", grad_variances)
np.save(prefix+"convergences"+spec+".npy",convergences)